In [ ]:
import numpy as np
import pandas as pd

In [ ]:
marco = input('Pathway of raw Marco learning data (get rid of quotations):   ')
marco = pd.read_excel(marco)

In [ ]:
marco

In [ ]:
overall = input('Pathway of raw Illuminate learning data (get rid of quotations):   ')
overall = pd.read_excel(overall)

In [ ]:
overall

In [ ]:
overall_standards = input('Pathway of raw Illuminate Standards learning data (get rid of quotations):   ')
overall_standards = pd.read_excel(overall_standards)

In [ ]:
marco = marco[['student_id', 'Total']]
marco

In [ ]:
marco = marco.fillna(0)
marco

In [ ]:
combined = pd.merge(overall, marco, how='left', left_on='Student Id', right_on='student_id')
combined

In [ ]:
combined = combined.drop_duplicates(subset=['Student Id'])
combined

In [ ]:
combined['Number Correct'] = combined.apply(lambda row: row['Total']
                            if pd.notna(row['Total']) else row['Number Correct'], axis=1)
combined

In [ ]:
combined['Performance Band Level'] = combined.apply(lambda row: row['Total']
                            if pd.notna(row['Total']) else row['Performance Band Level'], axis=1)
combined

In [ ]:
combined['Percentage'] = ((combined['Number Correct'] / combined['Number Possible']) * 100).round(2).astype(str) + '%'
combined

In [ ]:
combined['% Correct'] = combined['Percentage']
combined

In [ ]:
columns_to_drop = ['student_id', 'Total', 'Percentage']
final_overall = combined.drop(columns=columns_to_drop)
final_overall

In [ ]:
overall_standards

In [ ]:
combined

In [ ]:
columns_to_drop = ['Percentage']
combined = combined.drop(columns=columns_to_drop)

In [ ]:
combined

In [ ]:
combined['Percentage'] = ((combined['Number Correct'] / combined['Number Possible']) * 100).round(2)
combined

In [ ]:
combined['% Correct'] = combined['Percentage']
combined

In [ ]:
marco_standards = combined[['student_id', 'Performance Band Level', '% Correct']]
marco_standards

In [ ]:
overall_standards

In [ ]:
combined_standards = pd.merge(overall_standards, marco_standards, how='left', left_on='Student ID', right_on='student_id')
combined_standards

In [ ]:
melted = combined_standards.melt(['Current Site ID','Student ID', 'Current Grade Level', 'State Student ID',
             'First Name', 'Last Name', 'English Proficiency Code',
            'English Proficiency', 'Student Is Special Ed?', 'Date Taken', 'student_id',
            'Performance Band Level_y', '% Correct'], var_name='Standard', value_name='Level')
melted

In [ ]:
non_marco = melted[melted['student_id'].isna()]
non_marco

In [ ]:
marco_students = melted[melted['student_id'].notna()]
marco_students

In [ ]:
marco_students = marco_students[marco_students['Level'].notna()]
marco_students

In [ ]:
performance_band = marco_students[marco_students['Standard'].str.contains('Performance Band Level')]
performance_band

In [ ]:
performance_band['Level'] = performance_band['Performance Band Level_y']
performance_band

In [ ]:
percent_correct = marco_students[marco_students['Standard'].str.contains('Percent Correct')]
percent_correct

In [ ]:
percent_correct['Level'] = percent_correct['% Correct']
percent_correct

In [ ]:
mask = marco_students['Standard'].str.contains('Performance Band Level')
marco_students = marco_students[~mask]

In [ ]:
marco_students

In [ ]:
mask = marco_students['Standard'].str.contains('Percent Correct')
marco_students = marco_students[~mask]
marco_students

In [ ]:
marco_students = pd.concat([marco_students, performance_band, percent_correct])
marco_students

In [ ]:
non_marco

In [ ]:
full_students = pd.concat([non_marco, marco_students])
full_students

In [ ]:
columns_to_drop = ['student_id', 'Performance Band Level_y', '% Correct']
full_students = full_students.drop(columns=columns_to_drop)

In [ ]:
unmelted = full_students.groupby(['Student ID', 'Current Site ID', 'Current Grade Level', 'State Student ID',
             'First Name', 'Last Name', 'English Proficiency Code',
            'English Proficiency', 'Student Is Special Ed?', 'Date Taken', 'Standard'])['Level'].max().unstack()
unmelted

In [ ]:
unmelted = unmelted.rename(columns={'Performance Band Level_x': 'Performance Band Level'})
unmelted

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "CFA Overall with Marco", filename = "CFA Overall with Marco"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(final_overall)

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "CFA Standards ELA with Marco", filename = "CFA Standards ELA with Marco"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(unmelted)